In [191]:
import sympy as sym 
from IPython.display import display, Math
from itertools import islice # For non-continuous summation

hierarchy = ["\mathrm{s}", "\mathrm{p}", "\mathrm{m}", "\mathrm{sm}"]
hierarchy_names = ["star","planet", "moon", "submoon"]

def latex_this(expression):
    display(Math(sym.latex(expression)))


In [192]:
# If you want to find the spin frequency equation set this to true, to false, if you want to find the orbit frequency equation
find_spin_frequency_equation = False
j = 4 # hierarchy number of body to calculate equations for. 

In [193]:
index_abbreviation_disturbed_body = hierarchy[j-1]
name_of_disturbed_body = hierarchy_names[j-1]

if j-1==0:
    neighbour_array = [1]
elif j==len(hierarchy):
    neighbour_array = [-1]
else:
    neighbour_array = [-1,1]
    
index_abbreviations_of_all_disturbing_bodies = [hierarchy[j-1+const] for const in neighbour_array] # All index abbreviations that tidally influence j.
hierarchy_number_of_all_disturbing_bodies = [hierarchy.index(el)+1 for el in index_abbreviations_of_all_disturbing_bodies] # the actual associated hierarchy numbers 
names_of_all_disturbing_bodies = [hierarchy_names[i-1] for i in hierarchy_number_of_all_disturbing_bodies]


print()
print(f"Finding Equations for body in hierarchy number {j}, index abbreviation {index_abbreviation_disturbed_body}, standing for {name_of_disturbed_body}.")
print()
print(f"The {name_of_disturbed_body} is disturbed by bodies with hierarchy number",*hierarchy_number_of_all_disturbing_bodies,
f" which stand for ", *names_of_all_disturbing_bodies, " respectively. Abbreviated by ", *index_abbreviations_of_all_disturbing_bodies, ".")
print()


Finding Equations for body in hierarchy number 4, index abbreviation \mathrm{sm}, standing for submoon.

The submoon is disturbed by bodies with hierarchy number 3  which stand for  moon  respectively. Abbreviated by  \mathrm{m} .



In [194]:

k_val = min(hierarchy_number_of_all_disturbing_bodies) # k is the left neighbour of the disturbed body, i.e. the hierarchy number of the body
    # the disturbed body orbits.

# define coordinates that all derivatives are taken with respect to 
if not find_spin_frequency_equation:
    # orbit frequency equation is found
    list_of_q_j = [sym.symbols(r"\varphi_{"+f"{index_abbreviation_disturbed_body}-{hierarchy[k_val-1]}"+"}")] 
else:
    # spin frequency equation is found
    list_of_q_j = [sym.symbols(fr"\psi_{index_abbreviation_disturbed_body}")] 

print("All coordinates that derivatives are taken with respect to: ")
for i in list_of_q_j:
    latex_this(i)


All coordinates that derivatives are taken with respect to: 


<IPython.core.display.Math object>

In [195]:
# define the left hand side of the Euler-Lagrangian, dependent on whether the spin frequency or orbit frequency equation is looked for 
# For this firstly define necessary variables. 

# CONSTANTS 
m_j = sym.symbols(f"m_{index_abbreviation_disturbed_body}")
R_j = sym.symbols(f"R_{index_abbreviation_disturbed_body}")
k_2j = sym.symbols("k_{2"+f"{index_abbreviation_disturbed_body}"+"}")
G =  sym.symbols("\mathrm{G}")
alpha_j = sym.symbols(f"α_{index_abbreviation_disturbed_body}")
Q_j = sym.symbols(f"Q_{index_abbreviation_disturbed_body}")
I_j = alpha_j*m_j*R_j**2
a_i_j = sym.Function("a_{"+f"{index_abbreviation_disturbed_body}-{hierarchy[k_val-1]}"+"}")(t)

# TIME DEPENDENT VARIABLES
t = sym.symbols("t")
Omega_j = sym.Function(f"Ω_{index_abbreviation_disturbed_body}")(t)


# recursively defined quantities and functions of interest
i = sym.symbols('i', integer=True) # summing index (values later in sum: all disturbing bodies index abbreviations)
a = sym.IndexedBase('a_{'+f'{index_abbreviation_disturbed_body}'+'-}')
n = sym.IndexedBase('n_{'+f'{index_abbreviation_disturbed_body}'+'-}')
indices = hierarchy_number_of_all_disturbing_bodies
disturber_mass = sym.IndexedBase("m")

if len(indices)==1:
        # before taking the sum, properly define the lower and upper boundary by duplicating the indices array in necessary
        indices = indices*2

def signum(i):
        return sym.Function(r"\mathrm{sgn}")(Omega_j,n[i])

    
def kappa_constant(i):
    # dependance on j vanishes because those variables are in the global namespace
    return G*disturber_mass[i]**2*k_2j*R_j**5
    

def lambda_constant(i,k):
    # dependance on j vanishes because those variables are in the global namespace
    base_value = - sym.Rational(3,2) * kappa_constant(i) * 1 / Q_j * signum(i)
    if find_spin_frequency_equation:
        return base_value
    if not find_spin_frequency_equation:
        value = - base_value*sym.KroneckerDelta(i, k)
        return value

In [216]:
if find_spin_frequency_equation:
    
    left_hand_side_ELE = Omega_j.diff(t)*I_j
    right_hand_side_ELE = sym.Sum(lambda_constant(i,0) * a[i]**(-6), (i, indices)).doit() # k doesnt matter here

if not find_spin_frequency_equation:

    mu = sym.symbols("μ_{"+f"{index_abbreviation_disturbed_body}-{hierarchy[k_val-1]}"+"}")
    
    left_hand_side_ELE = 1/2 * m_j * ((mu)/(a_i_j))**(sym.Rational(1/2))*a_i_j.diff(t)
    right_hand_side_ELE = sym.Sum(lambda_constant(i,k=k_val) * a[i]**(-6), (i, indices)).doit()


In [217]:
if len(indices)>1:
    # remove those terms that are overcounting if there are two disturbers: (1,3) is treated as a range instead of actually summing over only 1 and 3
    right_hand_side_ELE = right_hand_side_ELE.subs({disturber_mass[i]: 0 for i in range(indices[0],indices[1]+1) if i not in indices})

print()
print("Left Hand Side of ELE")
latex_this(left_hand_side_ELE)
print()
print("Corrected Right Hand Side of ELE")
latex_this(right_hand_side_ELE)
print()
print("Setting Equal and Solving . . .")
print()
    


Left Hand Side of ELE


<IPython.core.display.Math object>


Corrected Right Hand Side of ELE


<IPython.core.display.Math object>


Setting Equal and Solving . . .



In [220]:
equation = sym.Eq(left_hand_side_ELE, right_hand_side_ELE)
print("Equation to solve: ")
print()
latex_this(equation)
if find_spin_frequency_equation:
    solution = sym.solve(equation, Omega_j.diff(t))
    print()
    print()
    print("Solution:")
    display(Math(sym.latex(Omega_j.diff(t))+"="+sym.latex(solution[0])))
if not find_spin_frequency_equation:
    solution = sym.solve(equation, a_i_j.diff(t))
    tweaked_right_hand_side = solution[0].subs(G,mu/disturber_mass[k_val]) # Replace G with mu expression, approximating G(m_small + m_big)\approx G m_big 
    
    print()
    print()
    print("Solution:")
    display(Math(sym.latex(a_i_j.diff(t))+"="+sym.latex(tweaked_right_hand_side)))






Equation to solve: 



<IPython.core.display.Math object>



Solution:


<IPython.core.display.Math object>